In [ ]:
%pip install langchain langchain-openai langchain-community sqlalchemy

# RAG Tool

In [1]:
import sys
import os
import json

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

SRC_PATH = os.path.join(PROJECT_ROOT, 'src')

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
    print(f"Đã thêm '{SRC_PATH}' vào sys.path")
else:
    print(f"'{SRC_PATH}' đã có trong sys.path")


try:
    from retriever.smart_retriever import SmartRetriever
    print("Import SmartRetriever thành công.")
except ImportError as e:
    print(f"LỖI IMPORT: {e}")

Đã thêm 'd:\BookInsight\src' vào sys.path
t
Import SmartRetriever thành công.


In [2]:
# khởi tạo RAG
# load model BGE, FAISS index, kết nối SQL
print("Đang khởi tạo SmartRetriever...")
try: 
    smart_rag_engine = SmartRetriever(device = "cpu")
    print("\nSmartRetriever đã sẵn sàng")
except Exception as e:
    print("Lỗi khởi tạo SmartRetriever")

Đang khởi tạo SmartRetriever...
[SmartRetriever] Đang khởi tạo các thành phần...
[VectorStore] Đang khởi tạo...
[VectorStore] Đang tải Model Embedding: BAAI/bge-small-en-v1.5
[VectorStore] Đang tải FAISS index từ: d:\BookInsight\data\vectorstores\index.faiss
[VectorStore] Đang tải metadata từ: d:\BookInsight\data\vectorstores\unique_ids.pkl
 [VectorStore] Khởi tạo hoàn tất. Sẵn sàng tìm kiếm.
[SQLDatabase] Đang kết nối tới: d:\BookInsight\data\database\books.db
[SQLDatabase] Kết nối thành công.
[QueryExpander] Đã khởi tạo với model: gpt-3.5-turbo
[SmartRetriever] Khởi tạo hoàn tất.

SmartRetriever đã sẵn sàng


In [3]:
from langchain_core.tools import BaseTool
from pydantic import BaseModel, Field, PrivateAttr
import json
from typing import Type, Any

# 1) Định nghĩa input cho Tool
class SmartRAGInput(BaseModel):
    query: str = Field(description="Một câu hỏi hoặc chủ đề để tìm kiếm sách.")

# 2) Định nghĩa Tool
class SmartRAGTool(BaseTool):
    """
    Một Tool cho phép Agent sử dụng SmartRetriever (RAG-Fusion)
    để tìm kiếm thông tin cho sách dựa trên ngữ nghĩa.
    """
    name: str = "smart_rag_engine"
    description: str = (
        "Rất hữu ích khi cần trả lời câu hỏi về nội dung sách, mô tả sách, "
        "tiểu sử tác giả, gợi ý sách theo chủ đề."
    )
    args_schema: Type[BaseModel] = SmartRAGInput

    _rag_engine: Any = PrivateAttr()

    def __init__(self, rag_engine, **data):
        super().__init__(**data)
        self._rag_engine = rag_engine

    def _run(self, query: str) -> str:
        print(f"\n[SmartRAGTool] đang chạy với câu hỏi: {query}")
        try:
            results = self._rag_engine.retrieve(query, top_k=3)
            return json.dumps(results, indent=2, ensure_ascii=False)
        except Exception as e:
            return f"Lỗi khi chạy SmartRAGTool: {e}"

    async def _arun(self, query: str) -> str:
        return self._run(query)

# Khởi tạo tool với instance đã tạo ở Cell 2
smart_rag_tool = SmartRAGTool(smart_rag_engine)
print("Đã định nghĩa và khởi tạo SmartRAGTool")

Đã định nghĩa và khởi tạo SmartRAGTool


In [4]:
# Chạy Thử Tool 
print("\n--- Chạy thử Tool với câu hỏi: 'python programming books' ---")
try:
    # (Giả sử 'smart_rag_tool' đã được khởi tạo ở cell ngay trên)
    tool_result = smart_rag_tool.run("python programming books")
    
    print("\nKẾT QUẢ TỪ TOOL")
    print(tool_result)
    
except Exception as e:
    print(f"Lỗi khi chạy Tool: {e}")


--- Chạy thử Tool với câu hỏi: 'python programming books' ---

[SmartRAGTool] đang chạy với câu hỏi: python programming books

[SmartRetriever] Bắt đầu truy vấn cho: 'python programming books'
[QueryExpander] Đang mở rộng câu hỏi: 'python programming books'...
-> [QueryExpander] Các câu hỏi đã mở rộng: ['python programming books', 'best python programming books', 'python coding books', 'top python programming guides', 'recommended python programming resources']
[SmartRetriever] Đang thực hiện tìm kiếm song song...
[SmartRetriever] Đang hợp nhất kết quả với RRF...
[SmartRetriever] Truy vấn RAG-Fusion hoàn tất.

KẾT QUẢ TỪ TOOL
[
  {
    "unique_id": "1119787602",
    "title": "Python All-in-One For Dummies (For Dummies (Computer/Tech))",
    "author_name": "Alan Simpson",
    "publisher": "For Dummies; 2nd edition (April 9, 2021)",
    "publication_year": -1.0,
    "main_category": "Books",
    "categories": "Books > Computers & Technology > Programming Languages",
    "book_format": "

# Text-to-SQL

In [5]:
from langchain_openai import ChatOpenAI
try:
    # dùng gpt-4o để tạo agent, để tiết kiệm có thể dùng 'gpt-3.5-turbo'
    llm = ChatOpenAI(model = 'gpt-4o', temperature = 0)
    print("Khởi tạo LLM gpt-4o thành công")
except Exception as e:
    print(f"Lỗi khi khởi tạo LLM")
    print("Đảm bảo OPENAI_API_KEY được đặt")

   



Khởi tạo LLM gpt-4o thành công


In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool, InfoSQLDatabaseTool
import os
import re

try: 
    from utils.config import SQL_DB_PATH
except ImportError as e:
    print(f"Lỗi import config (sẽ dùng path thủ công): {e}")
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
    SQL_DB_PATH = os.path.join(PROJECT_ROOT, "data", "database", "books.db")

print(f"Đang kết nối Langchain với SQL DB tại: {SQL_DB_PATH}")
db_uri= f"sqlite:///{SQL_DB_PATH}"
db = SQLDatabase.from_uri(db_uri)

sql_tool = QuerySQLDatabaseTool(db=db, llm=llm)

sql_tool.description = (
    "Rất hưu ích khi bạn cần trả lời các câu hỏi về dữ liệu CÓ CẤU TRÚC (structured data) "
    "như giá (price), xếp hạng (rating), số trang(page_count), năm xuất bản (publication_year), "
    "hoặc để đếm (count), tính trung bình (average), tìm giá trị lớn nhất (max)/nhỏ nhất (min). "
    "KHÔNG dùng Tool này để hỏi về mô tả sách, nội dung, hay tiểu sử tác giả. "
)

print("SQLDatabaseTool đã được khởi tạo thành công")

from langchain.chains import create_sql_query_chain

print("\n--- Chạy thử Text-to-SQL: 'Which book has the highest price?' ---")
try:
    # 1) Sinh câu lệnh SQL từ câu hỏi tự nhiên
    nl2sql_chain = create_sql_query_chain(llm, db)
    generated_sql = nl2sql_chain.invoke({"question": "How many books are written by 'Unknown'?"})
    print("\n[Generated SQL]\n", generated_sql)

    # 2) Làm sạch SQL: bỏ code fences, tiền tố, lời bình; trích SELECT/WITH
    text = generated_sql.strip()
    # bỏ toàn bộ code fences
    text = text.replace("```sql", "").replace("```", "")
    # bỏ tiền tố SQLQuery:/SQL: xuất hiện bất kỳ đâu
    text = re.sub(r"(?i)\bSQLQuery:\s*", "", text)
    # bỏ mọi đoạn bình luận LLM như 'Assuming ...' và phần sau
    text = re.sub(r"(?is)Assuming[\s\S]*$", "", text).strip()
    # loại bỏ các dòng bắt đầu bằng dấu ngoặc '('
    text = "\n".join([ln for ln in text.splitlines() if not ln.strip().startswith("(")])

    # trích câu SELECT/WITH đầu tiên kết thúc bằng ;
    match = re.search(r"(?is)\bselect\b[\s\S]+?;", text)
    if not match:
        match = re.search(r"(?is)\bwith\b[\s\S]+?;", text)
    cleaned_sql = match.group(0).strip() if match else text

    print("\n[Cleaned SQL]\n", cleaned_sql)

    # 3) Thực thi SQL trên DB
    result = db.run(cleaned_sql)
    print("\n --KẾT QUẢ TRẢ LỜI-- ")
    print(result)
except Exception as e:
    print(f"Lỗi Text-to-SQL: {e}")



Lỗi import config (sẽ dùng path thủ công): No module named 'utils.config'
Đang kết nối Langchain với SQL DB tại: d:\BookInsight\data\database\books.db
SQLDatabaseTool đã được khởi tạo thành công

--- Chạy thử Text-to-SQL: 'Which book has the highest price?' ---

[Generated SQL]
 ```sql
SQLQuery: SELECT COUNT(*) FROM "books" WHERE "author_name" = 'Unknown'
```

[Cleaned SQL]
 SELECT COUNT(*) FROM "books" WHERE "author_name" = 'Unknown'

 --KẾT QUẢ TRẢ LỜI-- 
[(28585,)]


In [7]:
# NL -> SQL test cell
from langchain.chains import create_sql_query_chain
import re


def run_nl_sql(question: str):
    print(f"\n=== NL Question ===\n{question}")
    chain = create_sql_query_chain(llm, db)
    generated_sql = chain.invoke({"question": question})
    print("\n[Generated SQL]\n", generated_sql)

    # Clean SQL: remove code fences / prefixes / commentary, extract first SELECT/WITH ...;
    text = generated_sql.strip()
    text = text.replace("```sql", "").replace("```", "")
    text = re.sub(r"(?i)\bSQLQuery:\s*", "", text)
    # strip any trailing commentary like 'Assuming ...' and anything after
    text = re.sub(r"(?is)Assuming[\s\S]*$", "", text).strip()
    # drop lines that begin with '(' (LLM notes)
    text = "\n".join([ln for ln in text.splitlines() if not ln.strip().startswith("(")])
    match = re.search(r"(?is)\bselect\b[\s\S]+?;", text)
    if not match:
        match = re.search(r"(?is)\bwith\b[\s\S]+?;", text)
    cleaned_sql = match.group(0).strip() if match else text
    print("\n[Cleaned SQL]\n", cleaned_sql)

    try:
        result = db.run(cleaned_sql)
        print("\n-- RESULT --\n", result)
    except Exception as e:
        print("\n-- ERROR --\n", e)


# Sample tests
questions = [
    "Which 5 books have the highest average_rating?",
    "What is the average price of books published after 2015?",
    "How many books are in the 'Programming Languages' category?",
    "List the top 10 most expensive books with their title and price.",
    "What is the average page_count for books written in English?",
    "Show all books by 'Unknown' with title and publication_year.",
    "Which author has the most books in the database?",
    "What is the maximum, minimum, and average price of all books?",
]

for q in questions:
    run_nl_sql(q)




=== NL Question ===
Which 5 books have the highest average_rating?

[Generated SQL]
 ```sql
SELECT "title", "author_name", "average_rating"
FROM books
ORDER BY "average_rating" DESC
LIMIT 5;
```

[Cleaned SQL]
 SELECT "title", "author_name", "average_rating"
FROM books
ORDER BY "average_rating" DESC
LIMIT 5;

-- RESULT --
 [('Notes from a Kidwatcher', 'Yetta M. Goodman', 5.0), ('Writings from a Black Woman Living in the Land of the "Free": Strength, Power, Resilience', 'Unknown', 5.0), ("Child Development: A Practitioner's Guide:2nd (Second) edition", 'Unknown', 5.0), ('Statesmen of the Lost Cause', 'Unknown', 5.0), ('Sounds from the Heart: Learning to Listen to Girls', 'Maureen Barbieri', 5.0)]

=== NL Question ===
What is the average price of books published after 2015?

[Generated SQL]
 ```sql
SQLQuery: SELECT AVG("price") FROM books WHERE "publication_year" > 2015
```

[Cleaned SQL]
 SELECT AVG("price") FROM books WHERE "publication_year" > 2015

-- RESULT --
 [(3.817942386831276,

# Memory và Prompt cho Agent

In [19]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Gợi ý schema cho prompt (để Agent biết tên bảng/cột)
try:
    schema_hint = db.get_table_info()
except Exception:
    schema_hint = "(Không lấy được schema từ DB)"

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", (
            "Bạn là 'BookInsight', một trợ lý AI thông thái và chuyên về sách. "
            "Bạn có quyền truy cập vào 2 công cụ mạnh mẽ:\n"
            "1. 'smart_book_retriever' (RAG Tool): Dùng tool này cho các câu hỏi MỞ về nội dung, "
            "mô tả, tóm tắt, tiểu sử tác giả, gợi ý sách theo chủ đề.\n"
            "2. 'sql_info_tool' (Schema Tool) + 'sql_tool' (Query Tool): Với câu hỏi CÓ CẤU TRÚC, "
            "hãy DÙNG 'sql_info_tool' để xem schema (bảng/cột) trước nếu chưa chắc tên cột/bảng, "
            "sau đó mới gọi 'sql_tool' để thực thi truy vấn.\n"
            "\n QUY TẮC QUAN TRỌNG NHẤT :\n"
            "Nếu một câu hỏi phức tạp (ví dụ: 'tóm tắt cuốn sách đắt nhất...'), bạn KHÔNG ĐƯỢC gọi 2 tool cùng lúc.\n"
            "Bạn PHẢI:\n"
            "1) Dùng 'sql_tool' để tìm cuốn sách (ví dụ: tìm ID hoặc title).\n"
            "2) Nhận kết quả (ID/title) từ 'sql_tool'.\n"
            "3) Dùng 'smart_book_retriever' với kết quả đó để tóm tắt/nội dung.\n"
            "Luôn dùng kết quả của tool trước làm đầu vào cho tool sau nếu cần.\n"
            "\nSchema gợi ý (có thể rút gọn):\n" + schema_hint
        )),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

print("Khởi tạo thành công 'memory' và 'agent_prompt'. ")

Khởi tạo thành công 'memory' và 'agent_prompt'. 


In [20]:
from langchain.agents import create_openai_tools_agent, AgentExecutor

# đảm bảo có sql_info_tool (phòng khi chưa chạy cell khởi tạo)
try:
    sql_info_tool
except NameError:
    try:
        from langchain_community.tools.sql_database.tool import InfoSQLDatabaseTool
        sql_info_tool = InfoSQLDatabaseTool(db=db)
        sql_info_tool.description = (
            "Dùng để xem schema (bảng, cột, khóa). Hãy gọi tool này trước khi viết SQL "
            "nếu bạn chưa chắc tên bảng/cột."
        )
        print("Đã tạo nhanh sql_info_tool từ Agent cell.")
    except Exception as _e:
        print(f"Không thể tạo sql_info_tool: {_e}")

# tập hợp tool
tools = [smart_rag_tool, sql_info_tool, sql_tool]

# tạo agent
try: 
    agent = create_openai_tools_agent(llm, tools, agent_prompt)
    print("Tạo agent thành công")
except Exception as e:
    print(f"Lỗi khi tạo agent: {e}")

# tạo Agent Executor (chạy invoke kết hợp agent, tools, memory)
try:
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        memory=memory,
        verbose=True
    )
    print("Agent Executor được lắp ráp thành công")
    print("\nBắt đầu chat với agent")
except Exception as e:
    print(f"Lỗi khi tạo Agent Executor: {e}")

    

Tạo agent thành công
Agent Executor được lắp ráp thành công

Bắt đầu chat với agent


In [ ]:
# --- Test Agent (Thử nghiệm) ---

def chat(question):
    print(f"\n--- 👤 User: {question} ---")
    try:
        # Gọi Agent Executor với câu hỏi
        # Nó sẽ tự động dùng 'memory' và 'prompt'
        response = agent_executor.invoke({"input": question})
        
        print("\n---🤖 Agent (Final Answer): ---")
        print(response['output'])
        
    except Exception as e:
        print(f"Lỗi khi chạy Agent: {e}")

# --- Test 1: Câu hỏi RAG ---
chat("What are some good books about 'AI for beginners'?")


--- 👤 User: What are some good books about 'AI for beginners'? ---


> Entering new AgentExecutor chain...

Invoking: `smart_rag_engine` with `{'query': 'AI for beginners'}`



[SmartRAGTool] đang chạy với câu hỏi: AI for beginners

[SmartRetriever] Bắt đầu truy vấn cho: 'AI for beginners'
[QueryExpander] Đang mở rộng câu hỏi: 'AI for beginners'...
-> [QueryExpander] Các câu hỏi đã mở rộng: ['AI for beginners', 'Introduction to AI', 'AI fundamentals for novices', 'Getting started with AI', 'AI basics for newcomers']
[SmartRetriever] Đang thực hiện tìm kiếm song song...
[SmartRetriever] Đang hợp nhất kết quả với RRF...
[SmartRetriever] Truy vấn RAG-Fusion hoàn tất.
[
  {
    "unique_id": "048624864X",
    "title": "Introduction to Artificial Intelligence: Second, Enlarged Edition (Dover Books on Mathematics)",
    "author_name": "Philip C. Jackson",
    "publisher": "Dover; Second Edition, Enlarged (June 1, 1985)",
    "publication_year": -1.0,
    "main_category": "Books",
    "catego

In [11]:
# Test 2: Câu hỏi Test Trí nhớ (Memory) 
# Agent phải "nhớ" được chúng ta đang nói về sách "AI for beginners"
chat("Which of those books has the highest rating?")


--- 👤 User: Which of those books has the highest rating? ---


> Entering new AgentExecutor chain...

Invoking: `sql_db_query` with `{'query': "SELECT title, MAX(average_rating) as highest_rating FROM books WHERE title IN ('Introduction to Artificial Intelligence: Second, Enlarged Edition (Dover Books on Mathematics)', 'Introduction to Deep Learning (The MIT Press)', 'Python Machine Learning: Learn Python in a Week and Master It. An Hands-On Introduction to Artificial Intelligence Coding, a Project-Based Guide with Practical Exercises (7 Days Crash Course, Book 2)') GROUP BY title ORDER BY highest_rating DESC LIMIT 1;"}`


[('Introduction to Artificial Intelligence: Second, Enlarged Edition (Dover Books on Mathematics)', 4.2)]The book with the highest rating among those listed is **"Introduction to Artificial Intelligence: Second, Enlarged Edition (Dover Books on Mathematics)"** by Philip C. Jackson, with an average rating of 4.2.

> Finished chain.

---🤖 Agent (Final Answer): ---
The

In [17]:
# --- Test 3: Câu hỏi SQL (Đếm) ---
# Agent nên chọn SQL Tool
chat("How many books are written by 'Nick Stellino'?")


--- 👤 User: How many books are written by 'Nick Stellino'? ---


> Entering new AgentExecutor chain...

Invoking: `sql_db_query` with `{'query': "SELECT COUNT(*) FROM books WHERE author_name = 'Nick Stellino'"}`


[(2,)]There are 2 books written by Nick Stellino.

> Finished chain.

---🤖 Agent (Final Answer): ---
There are 2 books written by Nick Stellino.


In [22]:
# --- Test 4: Câu hỏi Phức tạp (Kết hợp) ---
# Agent phải:
# 1. Dùng SQL Tool để tìm "sách 'Machine Learning' đắt nhất"
# 2. Dùng RAG Tool để "tóm tắt" cuốn sách nó vừa tìm được
chat("What is the most expensive book about 'Machine Learning' and can you summarize it?")


--- 👤 User: What is the most expensive book about 'Machine Learning' and can you summarize it? ---


> Entering new AgentExecutor chain...

Invoking: `sql_db_query` with `{'query': "SELECT unique_id, title FROM books WHERE categories LIKE '%Machine Learning%' ORDER BY price DESC LIMIT 1;"}`


[('B00YFOIW3Y', "What Do You Think About Machines That Think?: Today's Leading Thinkers on the Age of Machine Intelligence")]
Invoking: `smart_rag_engine` with `{'query': "What Do You Think About Machines That Think?: Today's Leading Thinkers on the Age of Machine Intelligence"}`



[SmartRAGTool] đang chạy với câu hỏi: What Do You Think About Machines That Think?: Today's Leading Thinkers on the Age of Machine Intelligence

[SmartRetriever] Bắt đầu truy vấn cho: 'What Do You Think About Machines That Think?: Today's Leading Thinkers on the Age of Machine Intelligence'
[QueryExpander] Đang mở rộng câu hỏi: 'What Do You Think About Machines That Think?: Today's Leading Thinkers on the Age of Machi

In [23]:
# --- Test 7: Câu hỏi Test Trí nhớ (Memory) ---
# Agent phải "nhớ" được chúng ta vừa nói về cuốn sách 
# "What Do You Think About Machines That Think?..."

chat("Who is the author of that book?")


--- 👤 User: Who is the author of that book? ---


> Entering new AgentExecutor chain...
The author of the book "What Do You Think About Machines That Think?: Today's Leading Thinkers on the Age of Machine Intelligence" is John Brockman.

> Finished chain.

---🤖 Agent (Final Answer): ---
The author of the book "What Do You Think About Machines That Think?: Today's Leading Thinkers on the Age of Machine Intelligence" is John Brockman.


# SavePreferenceTool
Một công cụ cho phép nói "Tôi thích thể loại sách X" hoặc "Tác giả Y là tác giả yêu thích của tôi", và Agent sẽ lưu thông tin này lại

In [24]:
import sqlite3
import os

try: 
    from utils.config import SQL_DB_PATH
except ImportError:
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
    SQL_DB_PATH = os.path.join(PROJECT_ROOT, "data", "database", "books.db")

print(f"Đang kết nối tới DB: {SQL_DB_PATH}")

# tạo bảng (nếu chưa có) để lưu sở thích
create_table_sql = """
    CREATE TABLE IF NOT EXISTS user_preferences (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    preference_type TEXT NOT NULL, 
    preference_value TEXT NOT NULL,
    UNIQUE(user_id, preference_type, preference_value)
    );
"""

try: 
    conn = sqlite3.connect(SQL_DB_PATH)
    cursor = conn.cursor()
    cursor.execute(create_table_sql)
    conn.commit()
    print("Bảng 'user_preference' đã xong")
except Exception as e:
    print(f"Lỗi khi tạo bảng:{e}")



Đang kết nối tới DB: d:\BookInsight\data\database\books.db
Bảng 'user_preference' đã xong


In [27]:
from langchain_core.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type 

# Định nghĩa Input cho Tool
class SavePreferenceInput(BaseModel):
    preference_type: str = Field(description="Loại sở thích (ví dụ: 'author', 'category', 'topic').")
    preference_value: str = Field(description="Giá trị của sở thích (ví dụ: 'Robert C.Martin', 'Science Fiction').")

class SavePreferenceTool(BaseTool):
    """
    Một Tool cho phép Agent LƯU LẠI sở thích của người dùng (như tác giả yêu thích, thể loại yêu thích,...) vào database.
    """
    name: str = "save_user_preference"
    description: str = (
        "Rất hữu ích khi người dùng BÀY TỎ một sở thích cụ thể. "
        "Dùng Tool này để GHI NHỚ tác giả yêu thích, thể loại yêu thích, ..."
        "hoặc chủ đề mà người dùng quan tâm."
    )
    args_schema: Type[BaseModel] = SavePreferenceInput
    db: SQLDatabase = Field(default_factory= lambda: SQLDatabase.from_uri(db_uri))

    def _run(self, preference_type: str, preference_value: str) -> str:
        """ Chạy tool khi Agent gọi. """
        print(f"\n[SavePreferenceTool] Đang lưu: {preference_type} = {preference_value}")
        user_id = 1 #mặc định cho user_id = 1

        try:
            insert_sql = "INSERT OR IGNORE INTO user_preferences (user_id, preference_type, preference_value) VALUES (?, ?, ?)"
            conn = sqlite3.connect(SQL_DB_PATH)
            cursor = conn.cursor()
            cursor.execute(insert_sql, (user_id, preference_type, preference_value))
            conn.commit()
            conn.close()

            return f"Đã lưu thành công sở thích :{preference_type} = {preference_value}."
        except Exception as e:
            return f"Lỗi khi lưu sở thích: {e}"

    async def _arun(self, preference_type: str, preference_value: str) -> str:
        return self._run(preference_type, preference_value)

print("Đã định nghĩa class SavePreferenceTool.")

Đã định nghĩa class SavePreferenceTool.


In [28]:
import sqlite3

# --- Tìm 5 tác giả có nhiều sách nhất để làm mẫu test ---
print("--- 5 tác giả có nhiều sách nhất trong DB ---")
try:
    conn = sqlite3.connect(SQL_DB_PATH)
    cursor = conn.cursor()
    # Đếm số sách của mỗi tác giả và sắp xếp giảm dần
    query = "SELECT author_name, COUNT(*) as book_count FROM books GROUP BY author_name ORDER BY book_count DESC LIMIT 5;"
    cursor.execute(query)
    top_authors = cursor.fetchall()
    conn.close()
    
    for author in top_authors:
        print(f"- {author[0]} ({author[1]} sách)")
        
    # Lấy tên tác giả đầu tiên để test
    # (Tác giả 'Unknown' thường nhiều nhất, nên chúng ta lấy người thứ 2)
    test_author_name = top_authors[1][0] 
    print(f"\n==> Sẽ dùng tác giả '{test_author_name}' để test.")
    
except Exception as e:
    print(f" Lỗi khi tìm tác giả: {e}")
    # Nếu lỗi, dùng tạm một giá trị
    test_author_name = "Philip C. Jackson"

--- 5 tác giả có nhiều sách nhất trong DB ---
- Unknown (28585 sách)
- James Patterson (69 sách)
- DK (53 sách)
- Stephen King (53 sách)
- Nora Roberts (50 sách)

==> Sẽ dùng tác giả 'James Patterson' để test.


In [29]:
# --- Khởi tạo Tool 3 ---
try:
    save_pref_tool = SavePreferenceTool()
    print("SavePreferenceTool đã được khởi tạo.")
    
    # --- Chạy Thử Tool với dữ liệu thật ---
    print(f"\n--- Chạy thử Tool: Lưu 'author' = '{test_author_name}' ---")
    
    tool_result = save_pref_tool.run({
        "preference_type": "author",
        "preference_value": test_author_name 
    })
    print(f"\n--- KẾT QUẢ TỪ TOOL ---: {tool_result}")
    
    # --- Kiểm tra trong DB ---
    print("\n--- Kiểm tra dữ liệu trong 'user_preferences' ---")
    conn = sqlite3.connect(SQL_DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM user_preferences")
    rows = cursor.fetchall()
    conn.close()
    
    for row in rows:
        print(row)
        
except Exception as e:
    print(f"Lỗi: {e}")

SavePreferenceTool đã được khởi tạo.

--- Chạy thử Tool: Lưu 'author' = 'James Patterson' ---

[SavePreferenceTool] Đang lưu: author = James Patterson

--- KẾT QUẢ TỪ TOOL ---: Đã lưu thành công sở thích :author = James Patterson.

--- Kiểm tra dữ liệu trong 'user_preferences' ---
(1, 1, 'author', 'James Patterson')


# GetPersonalizedRecommendationTool

In [34]:
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type, List
import sqlite3

# ddinhj nghĩa Input
class GetRecommendationInput(BaseModel):
    # tự động đọc từ DB
    pass

class GetPersonalizedRecommendationTool(BaseTool):
    """
    Một Tool cho phép agent GỢI Ý sách dựa trên SỞ THÍCH đã lưu của người dùng.
    Chỉ dùng Tool này khi người dùng hỏi gợi ý chung chung, ví dụ:
    'Gợi ý sách cho tôi', 'Bạn có đề xuất gì không?', 'Cuốn nào hay nhỉ?'
    """
    name: str = "get_personalized_recommendation"
    description: str = (
        "Rất hữu ích khi người dùng hỏi gợi ý sách CHUNG CHUNG (ví dụ: 'gợi ý cho tôi một cuốn sách chủ đề khoa học viễn tưởng'). "
        "Tool này sẽ tự động tìm sách dựa trên sở thích (tác giả, thể loại) đã lưu của người dùng. "
    )
    args_schema: Type[BaseModel] = GetRecommendationInput

    rag_tool: BaseTool

    def _run(self) -> str:
        """ Chạy tool khi Agent gọi """
        print(f"\n[GetRecommendationTool] ... Đang tìm sở thích đã lưu...")

        # đọc sở thích từ db
        user_id = 1
        preferences = []
        try:
            conn = sqlite3.connect(SQL_DB_PATH)
            cursor = conn.cursor()
            cursor.execute("SELECT preference_type, preference_value FROM user_preferences WHERE user_id = ?", (user_id,))
            rows = cursor.fetchall()
            conn.close()

            if not rows: 
                return "Tôi chưa biết sở thích của bạn. Hãy nói cho tôi biết bạn thích tác giả hoặc thể loại nào!"
            for row in rows:
                preferences.append(f"{row[0]} {row[1]}")
        except Exception as e:
            return f"Lỗi khi đọc sở thích: {e}"

        # tạo query mới nối tất cả sở thích lại thành 1 câu
        query_prompt = "Tìm sách dựa trên các sở thích sau: "+" AND ".join(preferences)
        print(f"[GetRecommendationTool] đã tạo query cá nhân hóa: '{query_prompt}'")

        print("[GetRecommendationTool] Đang gọi SmartRAGTool để tìm sách...")
        return self.rag_tool.run(query_prompt)

    async def _arun(self) -> str:
        return self._run()

print("Đã định nghĩa class GetPersonalizedRecommendationTool.")



Đã định nghĩa class GetPersonalizedRecommendationTool.


In [35]:
# Khởi tạo Tool 4 
try:
    
    get_rec_tool = GetPersonalizedRecommendationTool(rag_tool=smart_rag_tool)
    print("GetPersonalizedRecommendationTool đã được khởi tạo.")
    
    print("\nChạy thử Tool: 'Gợi ý sách cho tôi' ---")
    # Chúng ta không cần truyền gì vào, nó sẽ tự đọc DB
    tool_result = get_rec_tool.run({}) 
    
    print("\nKẾT QUẢ TỪ TOOL (CÁ NHÂN HÓA)")
    print(tool_result)
        
except Exception as e:
    print(f"Lỗi: {e}")

GetPersonalizedRecommendationTool đã được khởi tạo.

Chạy thử Tool: 'Gợi ý sách cho tôi' ---

[GetRecommendationTool] ... Đang tìm sở thích đã lưu...
[GetRecommendationTool] đã tạo query cá nhân hóa: 'Tìm sách dựa trên các sở thích sau: author James Patterson'
[GetRecommendationTool] Đang gọi SmartRAGTool để tìm sách...

[SmartRAGTool] đang chạy với câu hỏi: Tìm sách dựa trên các sở thích sau: author James Patterson

[SmartRetriever] Bắt đầu truy vấn cho: 'Tìm sách dựa trên các sở thích sau: author James Patterson'
[QueryExpander] Đang mở rộng câu hỏi: 'Tìm sách dựa trên các sở thích sau: author James Patterson'...
-> [QueryExpander] Các câu hỏi đã mở rộng: ['Tìm sách dựa trên các sở thích sau: author James Patterson', 'Find books based on the following interests: writer James Patterson', 'Discover novels by author James Patterson based on preferences', 'Explore books by James Patterson based on hobbies', 'Locate publications based on the following preferences: author James Patterson']

## 4 tools:
- smart_rag_tool: tìm kiếm ngữ nghĩa
- sql_tool: tìm kiếm cấu trúc
- save_pref_tool: lưu sở thích
- get_rec_tool: gợi ý cá nhân hóa

# Lắp ráp agent mở rộng


In [36]:
# Cập nhật Prompt (Dạy Agent về 2 tool mới) 
agent_prompt_final = ChatPromptTemplate.from_messages(
    [
        ("system", (
            "Bạn là 'BookInsight', một trợ lý AI thông thái và cá nhân hóa. "
            "Bạn phải luôn suy nghĩ TỪNG BƯỚC MỘT.\n"
            
            "Bạn có 4 công cụ:\n"
            "1. 'smart_book_retriever' (RAG Tool): Dùng cho câu hỏi MỞ (tóm tắt, nội dung, tiểu sử, gợi ý theo chủ đề).\n"
            "2. 'sql_tool' (SQL Tool): Dùng cho câu hỏi CỤ THỂ (giá, rating, đếm, so sánh, max/min).\n"
            "3. 'save_user_preference' (Save Tool): CHỈ dùng tool này khi người dùng BÀY TỎ SỞ THÍCH (ví dụ: 'tôi thích...', 'tác giả yêu thích của tôi là...').\n"
            "4. 'get_personalized_recommendation' (Rec Tool): CHỈ dùng tool này khi người dùng hỏi GỢI Ý CHUNG CHUNG (ví dụ: 'gợi ý sách cho tôi', 'tìm sách hay đi').\n"
            
            " QUY TẮC QUAN TRỌNG NHẤT:\n"
            "Nếu một câu hỏi phức tạp (ví dụ: 'tóm tắt cuốn sách đắt nhất...'), "
            "bạn PHẢI dùng 'sql_tool' để tìm sách trước, sau đó dùng 'smart_book_retriever' để tóm tắt."
        )),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Lắp ráp Agent Cuối cùng 
print("Đang lắp ráp Agent cuối cùng với 4 tools...")
try:
    # (Lấy các tool đã khởi tạo ở các cell trước)
    tools_final = [smart_rag_tool, sql_tool, save_pref_tool, get_rec_tool]
    
    # Tạo Agent (Bộ não) với prompt mới
    agent_final = create_openai_tools_agent(llm, tools_final, agent_prompt_final)
    
    # Tạo Agent Executor (Bộ điều hành)
    # tạo một memory mới cho lần test cuối này
    memory_final = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    agent_executor_final = AgentExecutor(
        agent=agent_final, 
        tools=tools_final, 
        memory=memory_final, 
        verbose=True
    )
    print("Agent Executor Cuối cùng (4 Tools) đã sẵn sàng")
    
except Exception as e:
    print(f"Lỗi khi lắp ráp Agent cuối cùng: {e}")

Đang lắp ráp Agent cuối cùng với 4 tools...
Agent Executor Cuối cùng (4 Tools) đã sẵn sàng


In [37]:
# Tạo một hàm chat mới để gọi 'agent_executor_final'
def chat_final(question):
    print(f"\n--- 👤 User: {question} ---")
    try:
        response = agent_executor_final.invoke({"input": question})
        print("\n--- 🤖 Agent (Final Answer): ---")
        print(response['output'])
    except Exception as e:
        print(f"Lỗi khi chạy Agent: {e}")

# Test 1: Dạy Agent sở thích (Test Tool 3) ---
chat_final("My favorite category is 'Science Fiction'.")

# Test 2: Bắt Agent dùng sở thích đó (Test Tool 4) ---
chat_final("Can you recommend a book for me?")


--- 👤 User: My favorite category is 'Science Fiction'. ---


> Entering new AgentExecutor chain...

Invoking: `save_user_preference` with `{'preference_type': 'category', 'preference_value': 'Science Fiction'}`



[SavePreferenceTool] Đang lưu: category = Science Fiction
Đã lưu thành công sở thích :category = Science Fiction.I've saved your favorite category as "Science Fiction." If you need any book recommendations or information, feel free to ask!

> Finished chain.

--- 🤖 Agent (Final Answer): ---
I've saved your favorite category as "Science Fiction." If you need any book recommendations or information, feel free to ask!

--- 👤 User: Can you recommend a book for me? ---


> Entering new AgentExecutor chain...

Invoking: `get_personalized_recommendation` with `{}`



[GetRecommendationTool] ... Đang tìm sở thích đã lưu...
[GetRecommendationTool] đã tạo query cá nhân hóa: 'Tìm sách dựa trên các sở thích sau: author James Patterson AND category Science Fiction'
[GetRecommendationTool